In [ ]:
import hist
import time
import json
import dask
import dill 
import operator
import numba
import numpy as np
import pandas as pd
from functools import reduce
from pprint import pprint
from itertools import combinations, combinations_with_replacement
from collections import defaultdict

import awkward as ak
import dask_awkward as dak
import hist.dask as hda
import vector
vector.register_awkward()

import matplotlib.colors as colors
import matplotlib.pyplot as plt
import mplhep as hep
import matplotlib.backends.backend_pdf
from matplotlib.backends.backend_pdf import PdfPages

from iEtaiPhiBinning import iEtaiPhiBinCollection

plt.style.use('seaborn-v0_8-white')
plt.rcParams['figure.dpi'] = 200
hep.style.use('CMS')

In [ ]:
def getHist(dataDict, mapList):
    return reduce(operator.getitem, mapList, dataDict)

def draw_thresholds(ax, threshold_map):
    for th in threshold_map.values():
        ax.axvline(x=th['score'], 
                   label=th['name'], 
                   color=th['color'],
                   linestyle='--',
                   linewidth=3,
        )

In [ ]:
import dill
with open(f'hist_result_Reco_2024I_ttbar_3.pkl', 'rb') as file:
    hist_result_reco = dill.load(file)
with open(f'hist_result_Scouting_2024I_ttbar_investigation15.pkl', 'rb') as file:
    hist_result_scouting = dill.load(file)

In [ ]:
hist_result_scouting

In [ ]:
import matplotlib as mpl
from cycler import cycler

mpl.rcParams['axes.prop_cycle'] = cycler(
    color=[
        '#3f90da', 
        '#ffa90e', 
        '#bd1f01', 
        '#94a4a2', 
        '#832db6', 
        '#a96b59', 
        '#e76300', 
        '#b9ac70', 
        '#717581', 
        '#92dadd'
    ]
)

In [ ]:
def draw_hist1d(hist_in, ax, trigger, label='', rebin=1, norm=False):
    
    hist_in = hist_in[
        :, 
        trigger, 
        hist.rebin(rebin),
    ]
    counts, _, bins = hist_in.to_numpy()
    
    if len(counts)>0: # check that there are events to plot
        _counts = counts[0]/(np.sum(counts[0])*np.diff(bins)) if norm else counts[0]
        errs = np.sqrt(counts[0])/(np.sum(counts[0])*np.diff(bins)) if norm else np.sqrt(counts[0])
        _errs = np.where(_counts==0, 0, errs)
        bin_centres = 0.5*(bins[1:] + bins[:-1])
        l = ax.errorbar(x=bin_centres,y=_counts,yerr=_errs,linestyle='')
        color = l[0].get_color()
        ax.errorbar(x=bins[:-1],y=_counts,drawstyle='steps-post',label=label,color=color)
        
    else:        
        l = ax.errorbar(x=[],y=[],yerr=[],drawstyle='steps-post') # plot nothing
        color = l[0].get_color()
        ax.errorbar(x=[],y=[],drawstyle='steps-post',label=label,color=color)

def draw_hist1d_ptrange(hist_in, ax, trigger, pt_low, pt_high, label='', rebin=1, norm=False):
    
    hist_in = hist_in[
        :, 
        trigger, 
        hist.rebin(rebin),
        hist.loc(pt_low) : hist.loc(pt_high),
    ]
    counts, _, bins = hist_in.to_numpy()
    
    if len(counts)>0: # check that there are events to plot
        _counts = counts[0]/(np.sum(counts[0])*np.diff(bins)) if norm else counts[0]
        errs = np.sqrt(counts[0])/(np.sum(counts[0])*np.diff(bins)) if norm else np.sqrt(counts[0])
        _errs = np.where(_counts==0, 0, errs)
        bin_centres = 0.5*(bins[1:] + bins[:-1])
        l = ax.errorbar(x=bin_centres,y=_counts,yerr=_errs,linestyle='')
        color = l[0].get_color()
        ax.errorbar(x=bins[:-1],y=_counts,drawstyle='steps-post',label=label,color=color)
        
    else:        
        l = ax.errorbar(x=[],y=[],yerr=[],drawstyle='steps-post') # plot nothing
        color = l[0].get_color()
        ax.errorbar(x=[],y=[],drawstyle='steps-post',label=label,color=color)
        
def rebin_to_i(counts, weighted_counts, errs, i_bins):
    
    sorted_indices = np.argsort(i_bins)
    i_bins = i_bins[sorted_indices]
    counts = counts[sorted_indices, :]
    weighted_counts = weighted_counts[sorted_indices]
    errs = errs[sorted_indices]
    
    unique_i, indices = np.unique(i_bins, return_index=True)
    
    # Initialize new arrays
    rebinned_counts = np.zeros(len(unique_i) - 1)
    rebinned_errs = np.zeros(len(unique_i) - 1)
    rebinned_weighted_counts = np.zeros(len(unique_i) - 1)
    
    # Iterate over unique iEta values and sum counts/errors
    for i in range(len(unique_i) - 1):
        start, end = indices[i], indices[i + 1]  # Get bin range
        rebinned_counts[i] = np.sum(counts[start:end])  # Sum counts
        rebinned_weighted_counts[i] = np.sum(weighted_counts[start:end])
        rebinned_errs[i] = np.sum(errs[start:end])  # Propagate errors

    return unique_i, rebinned_counts, rebinned_weighted_counts, rebinned_errs
        
def draw_hist1d_eta_ptweighted(hist_in, ax, trigger, label='', rebin=1, norm=False, use_ieta=False):
    
    if use_ieta: rebin = 1
    hist_selected = hist_in[
        :, 
        trigger, 
        hist.rebin(rebin),
        :,
    ]
    counts, _, eta_bins, pt_bins = hist_selected.to_numpy(flow=False)
    
    if use_ieta:
        ieta_bins = np.full_like(eta_bins, 0)
        bincollection = iEtaiPhiBinCollection()
        for i in range(len(eta_bins)):
            ieta_bins[i] = bincollection.iEta(eta_bins[i])
    
    pt_centers = 0.5 * (pt_bins[:-1] + pt_bins[1:])
    
    if len(counts)>0: # check that there are events to plot
        
        counts = np.squeeze(counts)
        
        weighted_counts = np.sum(counts * pt_centers, axis=1)
    
        weighted_variance = np.sum(counts * (pt_centers ** 2), axis=1)
        errs = np.sqrt(weighted_variance)
        errs = np.where(weighted_counts == 0, 0, errs)
        
        if use_ieta:
            ieta_bins, counts, weighted_counts, errs = rebin_to_i(counts, weighted_counts, errs, ieta_bins)
        
        if norm: 
            
            if use_ieta:
                scale_factor = np.sum(weighted_counts) * np.diff(ieta_bins)
            else:
                scale_factor = np.sum(weighted_counts) * np.diff(eta_bins)
            weighted_counts /= scale_factor
            errs /= scale_factor
        
        eta_centers = 0.5 * (eta_bins[1:] + eta_bins[:-1])
        if use_ieta:
            ieta_centers = 0.5 * (ieta_bins[1:] + ieta_bins[:-1])
            l = ax.errorbar(x=ieta_centers, y=weighted_counts, yerr=errs, linestyle='')
            color = l[0].get_color()
            ax.errorbar(x=ieta_bins[:-1], y=weighted_counts, drawstyle='steps-post', label=label, color=color)
        else:
            l = ax.errorbar(x=eta_centers, y=weighted_counts, yerr=errs, linestyle='')
            color = l[0].get_color()
            ax.errorbar(x=eta_bins[:-1], y=weighted_counts, drawstyle='steps-post', label=label, color=color)
            
    else:        
        l = ax.errorbar(x=[], y=[], yerr=[], drawstyle='steps-post')  # Plot nothing
        color = l[0].get_color()
        ax.errorbar(x=[], y=[], drawstyle='steps-post', label=label, color=color)
        
        
def draw_hist1d_phi_ptweighted(hist_in, ax, trigger, label='', rebin=1, norm=False, use_iphi=False):
    
    if use_iphi: rebin = 1
    hist_selected = hist_in[
        :, 
        trigger, 
        hist.rebin(rebin),
        :,
    ]
    counts, _, phi_bins, pt_bins = hist_selected.to_numpy(flow=False)
    
    if use_iphi:
        iphi_bins = np.full_like(phi_bins, 0)
        bincollection = iEtaiPhiBinCollection()
        for i in range(len(phi_bins)):
            iphi_bins[i] = bincollection.iPhi(phi_bins[i])
        print("phi_bins = ", phi_bins)
        print("iphi_bins = ", iphi_bins)
    
    pt_centers = 0.5 * (pt_bins[:-1] + pt_bins[1:])
    
    if len(counts)>0: # check that there are events to plot
        
        counts = np.squeeze(counts)
        
        weighted_counts = np.sum(counts * pt_centers, axis=1)
    
        weighted_variance = np.sum(counts * (pt_centers ** 2), axis=1)
        errs = np.sqrt(weighted_variance)
        errs = np.where(weighted_counts == 0, 0, errs)
        
        if use_iphi:
            iphi_bins, counts, weighted_counts, errs = rebin_to_i(counts, weighted_counts, errs, iphi_bins)
        
        if norm: 
            
            if use_iphi:
                scale_factor = np.sum(weighted_counts) * np.diff(iphi_bins)
            else:
                scale_factor = np.sum(weighted_counts) * np.diff(phi_bins)
            weighted_counts /= scale_factor
            errs /= scale_factor
        
        phi_centers = 0.5 * (phi_bins[1:] + phi_bins[:-1])
        if use_iphi:
            iphi_centers = 0.5 * (iphi_bins[1:] + iphi_bins[:-1])
            l = ax.errorbar(x=iphi_centers, y=weighted_counts, yerr=errs, linestyle='')
            color = l[0].get_color()
            ax.errorbar(x=iphi_bins[:-1], y=weighted_counts, drawstyle='steps-post', label=label, color=color)
        else:
            l = ax.errorbar(x=phi_centers, y=weighted_counts, yerr=errs, linestyle='')
            color = l[0].get_color()
            ax.errorbar(x=phi_bins[:-1], y=weighted_counts, drawstyle='steps-post', label=label, color=color)
            
    else:        
        l = ax.errorbar(x=[], y=[], yerr=[], drawstyle='steps-post')  # Plot nothing
        color = l[0].get_color()
        ax.errorbar(x=[], y=[], drawstyle='steps-post', label=label, color=color)
        

In [ ]:
histpath_scouting = ['Scouting_2024I', 'hists']
dataset_scouting = histpath_scouting[0]
title_scouting = 'Scouting 2024I'
triggers_scouting = [
    'DST_PFScouting_ZeroBias',
    'DST_PFScouting_AXONominal', 
    'DST_PFScouting_CICADAMedium', 
]
trigger_names_scouting = {
    'DST_PFScouting_AXOLoose': 'AXO Loose', 
    'DST_PFScouting_AXONominal': 'AXO Nominal', 
    'DST_PFScouting_AXOTight': 'AXO Tight', 
    'DST_PFScouting_AXOVLoose': 'AXO VLoose',
    'DST_PFScouting_AXOVTight': 'AXO VTight',
    'DST_PFScouting_CICADAMedium': 'CICADA Medium', 
    'DST_PFScouting_CICADATight': 'CICADA Tight', 
    'DST_PFScouting_CICADAVTight': 'CICADA VTight', 
    'DST_PFScouting_DoubleMuon': 'Double Muon',
    'DST_PFScouting_JetHT': 'Jet HT',
    'DST_PFScouting_ZeroBias': 'Zero Bias'
}

histpath_reco = ['Reco_2024I', 'hists']
dataset_reco = histpath_reco[0]
title_reco = 'JetMET 2024I'
triggers_reco = [
    'L1_ZeroBias', 
    'L1_AXO_Nominal', 
    'L1_CICADA_Medium'
]
trigger_names_reco = {
    'L1_ZeroBias': 'Zero Bias', 
    'L1_AXO_Nominal': 'AXO Nominal', 
    'L1_CICADA_Medium': 'CICADA Medium'
}


# pT distributions

In [ ]:
def draw_hist_pt(hist_in, ax, trigger, label='', rebin=1, norm=False):
    
    hist_in = hist_in[:, trigger, :, :, hist.rebin(rebin)]
    hist_in = hist_in.project("pt")
    counts, bins = hist_in.to_numpy()
    
    if len(counts)>0: # check that there are events to plot
        _counts = counts/(np.sum(counts)*np.diff(bins)) if norm else counts
        errs = np.sqrt(counts)/(np.sum(counts)*np.diff(bins)) if norm else np.sqrt(counts)
        _errs = np.where(_counts==0, 0, errs)
        bin_centres = 0.5*(bins[1:] + bins[:-1])
        
        l = ax.errorbar(x=bin_centres,y=_counts,yerr=_errs,linestyle='')
        color = l[0].get_color()
        ax.errorbar(x=bins[:-1],y=_counts,drawstyle='steps-post',label=label,color=color)
        
    else:        
        l = ax.errorbar(x=[],y=[],yerr=[],drawstyle='steps-post') # plot nothing
        color = l[0].get_color()
        ax.errorbar(x=[],y=[],drawstyle='steps-post',label=label,color=color)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(24, 10))
hist_name = 'lead_scoutingjet_eta_phi'
norm = True

for trigger in triggers_scouting:
    hist_current = getHist(hist_result_scouting, histpath_scouting+[hist_name])
    
    draw_hist_pt(hist_current, 
                 ax[0], 
                 trigger, 
                 label=trigger_names_scouting[trigger], 
                 rebin=1, 
                 norm=norm)
    
hep.cms.label('', ax=ax[0], data=True, lumi=None, year=title_scouting, com=13.6)
ax[0].set_xlim(0,500)
ax[0].set_ylabel(f'Events{" [A.U.]" if norm else ""}', loc='top')
ax[0].set_xlabel(r'Lead ScoutingJet $p_T$ [GeV]', loc='right')
ax[0].legend();
ax[1].set_visible(False)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(24, 10))
hist_name = 'lead_l1jet_eta_phi'
norm = True

for trigger in triggers_scouting:
    hist_current = getHist(hist_result_scouting, histpath_scouting+[hist_name])
    
    draw_hist_pt(hist_current, 
                 ax[0], 
                 trigger, 
                 label=trigger_names_scouting[trigger], 
                 rebin=1, 
                 norm=norm)
    
hep.cms.label('', ax=ax[0], data=True, lumi=None, year=title_scouting, com=13.6)
ax[0].set_xlim(0,300)
ax[0].set_ylabel(f'Events{" [A.U.]" if norm else ""}', loc='top')
ax[0].set_xlabel(r'Lead L1Jet $p_T$ [GeV]', loc='right')
ax[0].legend();
ax[1].set_visible(False)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(24, 10))
hist_name = 'lead_recojet_eta_phi'
norm = True

for trigger in triggers_reco:
    hist_current = getHist(hist_result_reco, histpath_reco+[hist_name])
    
    draw_hist_pt(hist_current, 
                 ax[0], 
                 trigger, 
                 label=trigger_names_reco[trigger], 
                 rebin=1, 
                 norm=norm)
    
hep.cms.label('', ax=ax[0], data=True, lumi=None, year=title_reco, com=13.6)
ax[0].set_xlim(0,500)
ax[0].set_ylabel(f'Events{" [A.U.]" if norm else ""}', loc='top')
ax[0].set_xlabel(r'Lead Reco Jet $p_T$ [GeV]', loc='right')
ax[0].legend();
ax[1].set_visible(False)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(24, 10))
hist_name = 'lead_l1jet_eta_phi'
norm = True

for trigger in triggers_reco:
    hist_current = getHist(hist_result_reco, histpath_reco+[hist_name])
    
    draw_hist_pt(hist_current, 
                 ax[0], 
                 trigger, 
                 label=trigger_names_reco[trigger], 
                 rebin=1, 
                 norm=norm)
    
hep.cms.label('', ax=ax[0], data=True, lumi=None, year=title_reco, com=13.6)
ax[0].set_xlim(0,300)
ax[0].set_ylabel(f'Events{" [A.U.]" if norm else ""}', loc='top')
ax[0].set_xlabel(r'Lead L1Jet $p_T$ [GeV]', loc='right')
ax[0].legend();
ax[1].set_visible(False)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(24, 10))
hist_name = 'lead_scoutingelectron_eta_phi'
norm = True

for trigger in triggers_scouting:
    hist_current = getHist(hist_result_scouting, histpath_scouting+[hist_name])
    
    draw_hist_pt(hist_current, 
                 ax[0], 
                 trigger, 
                 label=trigger_names_scouting[trigger], 
                 rebin=1, 
                 norm=norm)
    
hep.cms.label('', ax=ax[0], data=True, lumi=None, year=title_scouting, com=13.6)
ax[0].set_xlim(0,200)
ax[0].set_ylabel(f'Events{" [A.U.]" if norm else ""}', loc='top')
ax[0].set_xlabel(r'Lead ScoutingElectron $p_T$ [GeV]', loc='right')
ax[0].legend();
ax[1].set_visible(False)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(24, 10))
hist_name = 'lead_recoelectron_eta_phi'
norm = True

for trigger in triggers_reco:
    hist_current = getHist(hist_result_reco, histpath_reco+[hist_name])
    
    draw_hist_pt(hist_current, 
                 ax[0], 
                 trigger, 
                 label=trigger_names_reco[trigger], 
                 rebin=1, 
                 norm=norm)
    
hep.cms.label('', ax=ax[0], data=True, lumi=None, year=title_reco, com=13.6)
ax[0].set_xlim(0,200)
ax[0].set_ylabel(f'Events{" [A.U.]" if norm else ""}', loc='top')
ax[0].set_xlabel(r'Lead Reco Electron $p_T$ [GeV]', loc='right')
ax[0].legend();
ax[1].set_visible(False)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(24, 10))
hist_name = 'lead_scoutingphoton_eta_phi'
norm = True

for trigger in triggers_scouting:
    hist_current = getHist(hist_result_scouting, histpath_scouting+[hist_name])
    
    draw_hist_pt(hist_current, 
                 ax[0], 
                 trigger, 
                 label=trigger_names_scouting[trigger], 
                 rebin=1, 
                 norm=norm)
    
hep.cms.label('', ax=ax[0], data=True, lumi=None, year=title_scouting, com=13.6)
ax[0].set_xlim(0,200)
ax[0].set_ylabel(f'Events{" [A.U.]" if norm else ""}', loc='top')
ax[0].set_xlabel(r'Lead ScoutingPhoton $p_T$ [GeV]', loc='right')
ax[0].legend();
ax[1].set_visible(False)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(24, 10))
hist_name = 'lead_recophoton_eta_phi'
norm = True

for trigger in triggers_reco:
    hist_current = getHist(hist_result_reco, histpath_reco+[hist_name])
    
    draw_hist_pt(hist_current, 
                 ax[0], 
                 trigger, 
                 label=trigger_names_reco[trigger], 
                 rebin=1, 
                 norm=norm)
    
hep.cms.label('', ax=ax[0], data=True, lumi=None, year=title_reco, com=13.6)
ax[0].set_xlim(0,200)
ax[0].set_ylabel(f'Events{" [A.U.]" if norm else ""}', loc='top')
ax[0].set_xlabel(r'Lead Reco Photon $p_T$ [GeV]', loc='right')
ax[0].legend();
ax[1].set_visible(False)

# eta/phi maps (no pt-weighting)

In [ ]:
def draw_hist2d(hist_eta_phi, hist_ieta_iphi, trigger_name, object_name="", 
                dataset="Scouting", rebin_eta=1, rebin_phi=1, rebin_ieta=1, rebin_iphi=1):
    
    hist_eta_phi = hist_eta_phi[:,trigger,39:112,hist.rebin(rebin_phi),:]
    hist_eta_phi = hist_eta_phi[:, hist.rebin(rebin_eta), :, :]
    hist_eta_phi = hist_eta_phi.project("eta", "phi")
    counts_eta_phi, eta_bins, phi_bins = hist_eta_phi.to_numpy(flow=False)
    
    hist_ieta_iphi = hist_ieta_iphi[:,trigger,4:18,hist.rebin(rebin_iphi),:]
    hist_ieta_iphi = hist_ieta_iphi[:, hist.rebin(rebin_ieta), :, :]
    hist_ieta_iphi = hist_ieta_iphi.project("ieta", "iphi")
    counts_ieta_iphi, ieta_bins, iphi_bins = hist_ieta_iphi.to_numpy(flow=False)
    counts_ieta_iphi = np.roll(counts_ieta_iphi, shift=-8, axis=1)
        
    ieta_bin_centers = np.arange(counts_ieta_iphi.shape[0]) + 4.5
    iphi_bin_centers = np.arange(counts_ieta_iphi.shape[1]) + 0.5
    ieta_bin_edges = np.arange(4,counts_ieta_iphi.shape[0] + 5)
    iphi_bin_edges = np.arange(counts_ieta_iphi.shape[1] + 1)
    iphi_labels = np.arange(counts_ieta_iphi.shape[1])
    ieta_labels = np.arange(counts_ieta_iphi.shape[0])+4
    
    fig, ax = plt.subplots(1, 2, figsize=(16, 8))

    ax[0].tick_params(axis='both', which='major', labelsize=12)
    ax[0].set_ylim(-3.14,3.14)
    x_label_0 = object_name + r" $\eta$"
    ax[0].set_xlabel(x_label_0)
    y_label_0 = object_name + r" $\phi$"
    ax[0].set_ylabel(y_label_0)
    
    mesh = ax[0].pcolormesh(eta_bins, phi_bins, counts_eta_phi.T, cmap="plasma", norm="log")
    
    
    ax[1].tick_params(axis='both', which='major', labelsize=12)
    ax[1].set_xticks(ieta_bin_centers)
    ax[1].set_xticklabels(ieta_labels)
    ax[1].set_xlim(4,18)
    ax[1].set_yticks(iphi_bin_centers)
    ax[1].set_yticklabels(iphi_labels)
    x_label_1 = object_name + r" i$\eta$"
    ax[1].set_xlabel(x_label_1)
    y_label_1 = object_name + r" i$\phi$"
    ax[1].set_ylabel(y_label_1)

    mesh = ax[1].pcolormesh(ieta_bin_edges, iphi_bin_edges, counts_ieta_iphi.T, cmap="plasma", norm="log")
    
    plt.suptitle(dataset + ": " + trigger_name)
    plt.subplots_adjust(hspace=0.5)

    fig.show()

In [ ]:
hist_name_eta_phi =  'lead_scoutingjet_eta_phi'
hist_name_ieta_iphi =  'lead_scoutingjet_ieta_iphi'

for trigger in triggers_scouting:
    
    trigger_name = trigger_names_scouting[trigger]
    hist_current_eta_phi = getHist(hist_result_scouting, histpath_scouting+[hist_name_eta_phi])
    hist_current_ieta_iphi = getHist(hist_result_scouting, histpath_scouting+[hist_name_ieta_iphi])
    
    draw_hist2d(hist_current_eta_phi, hist_current_ieta_iphi, 
                trigger_name, object_name="Scouting Jet", dataset="Scouting", 
                rebin_eta=1, rebin_phi=1, rebin_ieta=1, rebin_iphi=1)

In [ ]:
hist_name_eta_phi =  'lead_l1jet_eta_phi'
hist_name_ieta_iphi =  'lead_l1jet_ieta_iphi'

for trigger in triggers_scouting:
    
    trigger_name = trigger_names_scouting[trigger]
    hist_current_eta_phi = getHist(hist_result_scouting, histpath_scouting+[hist_name_eta_phi])
    hist_current_ieta_iphi = getHist(hist_result_scouting, histpath_scouting+[hist_name_ieta_iphi])
    
    draw_hist2d(hist_current_eta_phi, hist_current_ieta_iphi, 
                trigger_name, object_name="L1 Jet", dataset="Scouting",
                rebin_eta=1, rebin_phi=1, rebin_ieta=1, rebin_iphi=1)

In [ ]:
hist_name_eta_phi =  'lead_recojet_eta_phi'
hist_name_ieta_iphi =  'lead_recojet_ieta_iphi'

for trigger in triggers_reco:
    
    trigger_name = trigger_names_reco[trigger]
    hist_current_eta_phi = getHist(hist_result_reco, histpath_reco+[hist_name_eta_phi])
    hist_current_ieta_iphi = getHist(hist_result_reco, histpath_reco+[hist_name_ieta_iphi])
    
    draw_hist2d(hist_current_eta_phi, hist_current_ieta_iphi, 
                trigger_name, object_name="Reco Jet", dataset="JetMET",
                rebin_eta=1, rebin_phi=1, rebin_ieta=1, rebin_iphi=1)

In [ ]:
hist_name_eta_phi =  'lead_l1jet_eta_phi'
hist_name_ieta_iphi =  'lead_l1jet_ieta_iphi'

for trigger in triggers_reco:
    
    trigger_name = trigger_names_reco[trigger]
    hist_current_eta_phi = getHist(hist_result_reco, histpath_reco+[hist_name_eta_phi])
    hist_current_ieta_iphi = getHist(hist_result_reco, histpath_reco+[hist_name_ieta_iphi])
    
    draw_hist2d(hist_current_eta_phi, hist_current_ieta_iphi, 
                trigger_name, object_name="L1Jet", dataset="JetMET",
                rebin_eta=1, rebin_phi=1, rebin_ieta=1, rebin_iphi=1)

In [ ]:
hist_name_eta_phi =  'lead_scoutingelectron_eta_phi'
hist_name_ieta_iphi =  'lead_scoutingelectron_ieta_iphi'

for trigger in triggers_scouting:
    
    trigger_name = trigger_names_scouting[trigger]
    hist_current_eta_phi = getHist(hist_result_scouting, histpath_scouting+[hist_name_eta_phi])
    hist_current_ieta_iphi = getHist(hist_result_scouting, histpath_scouting+[hist_name_ieta_iphi])
    
    draw_hist2d(hist_current_eta_phi, hist_current_ieta_iphi, 
                trigger_name, object_name="Scouting Electron", dataset="Scouting", 
                rebin_eta=1, rebin_phi=1, rebin_ieta=1, rebin_iphi=1)

In [ ]:
hist_name_eta_phi =  'lead_recoelectron_eta_phi'
hist_name_ieta_iphi =  'lead_recoelectron_ieta_iphi'

for trigger in triggers_reco:
    
    trigger_name = trigger_names_reco[trigger]
    hist_current_eta_phi = getHist(hist_result_reco, histpath_reco+[hist_name_eta_phi])
    hist_current_ieta_iphi = getHist(hist_result_reco, histpath_reco+[hist_name_ieta_iphi])
    
    draw_hist2d(hist_current_eta_phi, hist_current_ieta_iphi, 
                trigger_name, object_name="Reco Electron", dataset="JetMET", 
                rebin_eta=1, rebin_phi=1, rebin_ieta=1, rebin_iphi=1)

In [ ]:
hist_name_eta_phi =  'lead_scoutingphoton_eta_phi'
hist_name_ieta_iphi =  'lead_scoutingphoton_ieta_iphi'

for trigger in triggers_scouting:
    
    trigger_name = trigger_names_scouting[trigger]
    hist_current_eta_phi = getHist(hist_result_scouting, histpath_scouting+[hist_name_eta_phi])
    hist_current_ieta_iphi = getHist(hist_result_scouting, histpath_scouting+[hist_name_ieta_iphi])
    
    draw_hist2d(hist_current_eta_phi, hist_current_ieta_iphi, 
                trigger_name, object_name="Scouting Photon", dataset="Scouting", 
                rebin_eta=1, rebin_phi=1, rebin_ieta=1, rebin_iphi=1)

In [ ]:
hist_name_eta_phi =  'lead_recophoton_eta_phi'
hist_name_ieta_iphi =  'lead_recophoton_ieta_iphi'

for trigger in triggers_reco:
    
    trigger_name = trigger_names_reco[trigger]
    hist_current_eta_phi = getHist(hist_result_reco, histpath_reco+[hist_name_eta_phi])
    hist_current_ieta_iphi = getHist(hist_result_reco, histpath_reco+[hist_name_ieta_iphi])
    
    draw_hist2d(hist_current_eta_phi, hist_current_ieta_iphi, 
                trigger_name, object_name="Reco Photon", dataset="JetMET", 
                rebin_eta=1, rebin_phi=1, rebin_ieta=1, rebin_iphi=1)

# pT Weighted eta distributions

In [ ]:
def draw_hist1d_eta_ptweighted(hist_eta_phi, hist_ieta_iphi, ax0, ax1, label="", 
                               norm=False, rebin_eta=1, rebin_ieta=1):
    
    hist_eta_phi = hist_eta_phi[:,trigger,hist.rebin(rebin_eta),:,:]
    hist_eta = hist_eta_phi.project("eta", "pt")
    counts_eta, eta_bins, pt_bins = hist_eta.to_numpy(flow=False)
    counts_eta = np.squeeze(counts_eta)
    
    pt_centers = 0.5 * (pt_bins[:-1] + pt_bins[1:])
    
    weighted_counts_eta = np.sum(counts_eta * pt_centers, axis=1)
    weighted_variance_eta = np.sum(counts_eta * (pt_centers ** 2), axis=1)
    errs_eta = np.sqrt(weighted_variance_eta)
    errs_eta = np.where(weighted_counts_eta == 0, 0, errs_eta)
    
    if norm:
        scale_factor = np.sum(weighted_counts_eta) * np.diff(eta_bins)
        weighted_counts_eta /= scale_factor
        errs_eta /= scale_factor
    
    eta_centers = 0.5 * (eta_bins[1:] + eta_bins[:-1])
    l = ax0.errorbar(x=eta_centers, y=weighted_counts_eta, yerr=errs_eta, linestyle='')
    color = l[0].get_color()
    ax0.errorbar(x=eta_bins[:-1], y=weighted_counts_eta, drawstyle='steps-post', label=label, color=color)
            
    hist_ieta_iphi = hist_ieta_iphi[:,trigger,hist.rebin(rebin_ieta),:,:]
    hist_ieta = hist_ieta_iphi.project("ieta", "pt")
    counts_ieta, ieta_bins, pt_bins = hist_ieta.to_numpy(flow=False)
    
    weighted_counts_ieta = np.sum(counts_ieta * pt_centers, axis=1)
    weighted_variance_ieta = np.sum(counts_ieta * (pt_centers ** 2), axis=1)
    errs_ieta = np.sqrt(weighted_variance_ieta)
    errs_ieta = np.where(weighted_counts_ieta == 0, 0, errs_ieta)
    
    if norm:
        scale_factor = np.sum(weighted_counts_ieta) * np.diff(ieta_bins)
        weighted_counts_ieta /= scale_factor
        errs_ieta /= scale_factor
    
    ieta_bin_centers = np.arange(len(weighted_counts_ieta)) + 0.5
    ieta_bin_edges = np.arange(len(weighted_counts_ieta) + 1)
    ieta_labels = np.arange(len(weighted_counts_ieta))
    
    l = ax1.errorbar(x=ieta_bin_centers, y=weighted_counts_ieta, yerr=errs_ieta, linestyle='')
    color = l[0].get_color()
    ax1.errorbar(x=ieta_bin_edges[:-1], y=weighted_counts_ieta, drawstyle='steps-post', label=label, color=color)
    
    ax1.tick_params(axis='both', which='major', labelsize=12)
    ax1.set_xticks(ieta_bin_centers)
    ax1.set_xticklabels(ieta_labels)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(24, 10))
hist_name_eta_phi =  'lead_scoutingjet_eta_phi'
hist_name_ieta_iphi =  'lead_scoutingjet_ieta_iphi'
norm = True

for trigger in triggers_scouting:
    hist_current_eta_phi = getHist(hist_result_scouting, histpath_scouting+[hist_name_eta_phi])
    hist_current_ieta_iphi = getHist(hist_result_scouting, histpath_scouting+[hist_name_ieta_iphi])
    
    draw_hist1d_eta_ptweighted(hist_current_eta_phi, hist_current_ieta_iphi, ax[0], ax[1], 
                               label=trigger_names_scouting[trigger], 
                               norm=norm, rebin_eta=1, rebin_ieta=1)
    
    
hep.cms.label('', ax=ax[0], data=True, lumi=None, year=title_scouting, com=13.6)
ax[0].set_xlim(-2.4,2.4)
ax[0].set_ylabel(f'Jet pT-Weighted Events{" [A.U.]" if norm else ""}', loc='top')
ax[0].set_xlabel(r'Lead Scouting Jet $\eta$', loc='right')
ax[0].legend();

hep.cms.label('', ax=ax[1], data=True, lumi=None, year=title_scouting, com=13.6)
ax[1].set_xlim(4,18)
ax[1].set_ylabel(f'Jet pT-Weighted Events{" [A.U.]" if norm else ""}', loc='top')
ax[1].set_xlabel(r'Lead Scouting Jet i$\eta$', loc='right')
ax[1].legend();

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(24, 10))
hist_name_eta_phi =  'lead_recojet_eta_phi'
hist_name_ieta_iphi =  'lead_recojet_ieta_iphi'
norm = True

for trigger in triggers_reco:
    hist_current_eta_phi = getHist(hist_result_reco, histpath_reco+[hist_name_eta_phi])
    hist_current_ieta_iphi = getHist(hist_result_reco, histpath_reco+[hist_name_ieta_iphi])
    
    draw_hist1d_eta_ptweighted(hist_current_eta_phi, hist_current_ieta_iphi, ax[0], ax[1], 
                               label=trigger_names_reco[trigger], 
                               norm=norm, rebin_eta=1, rebin_ieta=1)
    
    
hep.cms.label('', ax=ax[0], data=True, lumi=None, year=title_reco, com=13.6)
ax[0].set_xlim(-2.4,2.4)
ax[0].set_ylabel(f'Jet pT-Weighted Events{" [A.U.]" if norm else ""}', loc='top')
ax[0].set_xlabel(r'Lead Reco Jet $\eta$', loc='right')
ax[0].legend();

hep.cms.label('', ax=ax[1], data=True, lumi=None, year=title_reco, com=13.6)
ax[1].set_xlim(4,18)
ax[1].set_ylabel(f'Jet pT-Weighted Events{" [A.U.]" if norm else ""}', loc='top')
ax[1].set_xlabel(r'Lead Reco Jet i$\eta$', loc='right')
ax[1].legend();

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(24, 10))
hist_name_eta_phi =  'lead_l1jet_eta_phi'
hist_name_ieta_iphi =  'lead_l1jet_ieta_iphi'
norm = True

for trigger in triggers_scouting:
    hist_current_eta_phi = getHist(hist_result_scouting, histpath_scouting+[hist_name_eta_phi])
    hist_current_ieta_iphi = getHist(hist_result_scouting, histpath_scouting+[hist_name_ieta_iphi])
    
    draw_hist1d_eta_ptweighted(hist_current_eta_phi, hist_current_ieta_iphi, ax[0], ax[1], 
                               label=trigger_names_scouting[trigger], 
                               norm=norm, rebin_eta=1, rebin_ieta=1)
    
    
hep.cms.label('', ax=ax[0], data=True, lumi=None, year=title_scouting, com=13.6)
ax[0].set_xlim(-2.4,2.4)
ax[0].set_ylabel(f'Jet pT-Weighted Events{" [A.U.]" if norm else ""}', loc='top')
ax[0].set_xlabel(r'Lead L1Jet $\eta$', loc='right')
ax[0].legend();

hep.cms.label('', ax=ax[1], data=True, lumi=None, year=title_scouting, com=13.6)
ax[1].set_xlim(4,18)
ax[1].set_ylabel(f'Jet pT-Weighted Events{" [A.U.]" if norm else ""}', loc='top')
ax[1].set_xlabel(r'Lead L1Jet i$\eta$', loc='right')
ax[1].legend();

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(24, 10))
hist_name_eta_phi =  'lead_l1jet_eta_phi'
hist_name_ieta_iphi =  'lead_l1jet_ieta_iphi'
norm = True

for trigger in triggers_reco:
    hist_current_eta_phi = getHist(hist_result_reco, histpath_reco+[hist_name_eta_phi])
    hist_current_ieta_iphi = getHist(hist_result_reco, histpath_reco+[hist_name_ieta_iphi])
    
    draw_hist1d_eta_ptweighted(hist_current_eta_phi, hist_current_ieta_iphi, ax[0], ax[1], 
                               label=trigger_names_reco[trigger], 
                               norm=norm, rebin_eta=1, rebin_ieta=1)
    
    
hep.cms.label('', ax=ax[0], data=True, lumi=None, year=title_reco, com=13.6)
ax[0].set_xlim(-2.4,2.4)
ax[0].set_ylabel(f'Jet pT-Weighted Events{" [A.U.]" if norm else ""}', loc='top')
ax[0].set_xlabel(r'Lead L1Jet $\eta$', loc='right')
ax[0].legend();

hep.cms.label('', ax=ax[1], data=True, lumi=None, year=title_reco, com=13.6)
ax[1].set_xlim(4,18)
ax[1].set_ylabel(f'Jet pT-Weighted Events{" [A.U.]" if norm else ""}', loc='top')
ax[1].set_xlabel(r'Lead L1Jet i$\eta$', loc='right')
ax[1].legend();

# non-pT Weighted eta distributions

In [ ]:
def draw_hist_eta(hist_eta, hist_ieta, trigger, ax0, ax1, label='', 
                  norm=False, rebin_eta=1, rebin_ieta=1):
    
    hist_eta = hist_eta[:,trigger,hist.rebin(rebin_eta),:,:]
    hist_eta = hist_eta.project("eta")
    counts_eta, eta_bins = hist_eta.to_numpy(flow=False)
    errs_eta = np.sqrt(counts_eta)
    errs_eta = np.where(counts_eta==0, 0, errs_eta)
    if norm:
        scale_factor = np.sum(counts_eta) * np.diff(eta_bins)
        counts_eta /= scale_factor
        errs_eta /= scale_factor
    
    eta_bin_centers = 0.5 * (eta_bins[1:] + eta_bins[:-1])
    l = ax0.errorbar(x=eta_bin_centers,y=counts_eta,yerr=errs_eta,linestyle='')
    color = l[0].get_color()
    ax0.errorbar(x=eta_bins[:-1],y=counts_eta,drawstyle='steps-post',label=label,color=color)
    
    hist_ieta = hist_ieta[:,trigger,hist.rebin(rebin_ieta),:,:]
    hist_ieta = hist_ieta.project("ieta")
    counts_ieta, ieta_bins = hist_ieta.to_numpy(flow=False)
    errs_ieta = np.sqrt(counts_ieta)
    errs_ieta = np.where(counts_ieta==0, 0, errs_ieta)
    if norm:
        scale_factor = np.sum(counts_ieta) * np.diff(ieta_bins)
        counts_ieta /= scale_factor
        errs_ieta /= scale_factor
    
    ieta_bin_centers = np.arange(len(counts_ieta)) + 0.5
    ieta_bin_edges = np.arange(len(counts_ieta) + 1)
    ieta_labels = np.arange(len(counts_ieta))
    
    l = ax1.errorbar(x=ieta_bin_centers, y=counts_ieta, yerr=errs_ieta, linestyle='')
    color = l[0].get_color()
    ax1.errorbar(x=ieta_bin_edges[:-1], y=counts_ieta, drawstyle='steps-post', label=label, color=color)
    
    ax1.tick_params(axis='both', which='major', labelsize=12)
    ax1.set_xticks(ieta_bin_centers)
    ax1.set_xticklabels(ieta_labels)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(24, 10))
hist_name_eta_phi =  'lead_scoutingjet_eta_phi'
hist_name_ieta_iphi =  'lead_scoutingjet_ieta_iphi'
norm = True

for trigger in triggers_scouting:
    hist_current_eta_phi = getHist(hist_result_scouting, histpath_scouting+[hist_name_eta_phi])
    hist_current_ieta_iphi = getHist(hist_result_scouting, histpath_scouting+[hist_name_ieta_iphi])
    
    draw_hist_eta(hist_current_eta_phi, hist_current_ieta_iphi, trigger, ax[0], ax[1], 
                  label=trigger_names_scouting[trigger], 
                  norm=norm, rebin_eta=1, rebin_ieta=1)
    
    
hep.cms.label('', ax=ax[0], data=True, lumi=None, year=title_scouting, com=13.6)
ax[0].set_xlim(-2.4,2.4)
ax[0].set_ylabel(f'Events{" [A.U.]" if norm else ""}', loc='top')
ax[0].set_xlabel(r'Lead Scouting Jet $\eta$', loc='right')
ax[0].legend();

hep.cms.label('', ax=ax[1], data=True, lumi=None, year=title_scouting, com=13.6)
ax[1].set_xlim(4,18)
ax[1].set_ylabel(f'Events{" [A.U.]" if norm else ""}', loc='top')
ax[1].set_xlabel(r'Lead Scouting Jet i$\eta$', loc='right')
ax[1].legend();

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(24, 10))
hist_name_eta_phi =  'lead_recojet_eta_phi'
hist_name_ieta_iphi =  'lead_recojet_ieta_iphi'
norm = True

for trigger in triggers_reco:
    hist_current_eta_phi = getHist(hist_result_reco, histpath_reco+[hist_name_eta_phi])
    hist_current_ieta_iphi = getHist(hist_result_reco, histpath_reco+[hist_name_ieta_iphi])
    
    draw_hist_eta(hist_current_eta_phi, hist_current_ieta_iphi, trigger, ax[0], ax[1], 
                  label=trigger_names_reco[trigger], 
                  norm=norm, rebin_eta=1, rebin_ieta=1)
    
    
hep.cms.label('', ax=ax[0], data=True, lumi=None, year=title_reco, com=13.6)
ax[0].set_xlim(-2.4,2.4)
ax[0].set_ylabel(f'Events{" [A.U.]" if norm else ""}', loc='top')
ax[0].set_xlabel(r'Lead Reco Jet $\eta$', loc='right')
ax[0].legend();

hep.cms.label('', ax=ax[1], data=True, lumi=None, year=title_reco, com=13.6)
ax[1].set_xlim(4,18)
ax[1].set_ylabel(f'Events{" [A.U.]" if norm else ""}', loc='top')
ax[1].set_xlabel(r'Lead Reco Jet i$\eta$', loc='right')
ax[1].legend();

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(24, 10))
hist_name_eta_phi =  'lead_l1jet_eta_phi'
hist_name_ieta_iphi =  'lead_l1jet_ieta_iphi'
norm = True

for trigger in triggers_scouting:
    hist_current_eta_phi = getHist(hist_result_scouting, histpath_scouting+[hist_name_eta_phi])
    hist_current_ieta_iphi = getHist(hist_result_scouting, histpath_scouting+[hist_name_ieta_iphi])
    
    draw_hist_eta(hist_current_eta_phi, hist_current_ieta_iphi, trigger, ax[0], ax[1], 
                  label=trigger_names_scouting[trigger], 
                  norm=norm, rebin_eta=1, rebin_ieta=1)
    
    
hep.cms.label('', ax=ax[0], data=True, lumi=None, year=title_scouting, com=13.6)
ax[0].set_xlim(-2.4,2.4)
ax[0].set_ylabel(f'Events{" [A.U.]" if norm else ""}', loc='top')
ax[0].set_xlabel(r'Lead L1Jet $\eta$', loc='right')
ax[0].legend();

hep.cms.label('', ax=ax[1], data=True, lumi=None, year=title_scouting, com=13.6)
ax[1].set_xlim(4,18)
ax[1].set_ylabel(f'Events{" [A.U.]" if norm else ""}', loc='top')
ax[1].set_xlabel(r'Lead L1Jet i$\eta$', loc='right')
ax[1].legend();

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(24, 10))
hist_name_eta_phi =  'lead_l1jet_eta_phi'
hist_name_ieta_iphi =  'lead_l1jet_ieta_iphi'
norm = True

for trigger in triggers_reco:
    hist_current_eta_phi = getHist(hist_result_reco, histpath_reco+[hist_name_eta_phi])
    hist_current_ieta_iphi = getHist(hist_result_reco, histpath_reco+[hist_name_ieta_iphi])
    
    draw_hist_eta(hist_current_eta_phi, hist_current_ieta_iphi, trigger, ax[0], ax[1], 
                  label=trigger_names_reco[trigger], 
                  norm=norm, rebin_eta=1, rebin_ieta=1)
    
    
hep.cms.label('', ax=ax[0], data=True, lumi=None, year=title_reco, com=13.6)
ax[0].set_xlim(-2.4,2.4)
ax[0].set_ylabel(f'Events{" [A.U.]" if norm else ""}', loc='top')
ax[0].set_xlabel(r'Lead L1Jet $\eta$', loc='right')
ax[0].legend();

hep.cms.label('', ax=ax[1], data=True, lumi=None, year=title_reco, com=13.6)
ax[1].set_xlim(4,18)
ax[1].set_ylabel(f'Events{" [A.U.]" if norm else ""}', loc='top')
ax[1].set_xlabel(r'Lead L1Jet i$\eta$', loc='right')
ax[1].legend();

# pT Weighted phi distributions

In [ ]:
def draw_hist1d_phi_ptweighted(hist_eta_phi, hist_ieta_iphi, ax0, ax1, trigger, label="", 
                               norm=False, rebin_phi=1, rebin_iphi=1):
    
    hist_eta_phi = hist_eta_phi[:,trigger,:,hist.rebin(rebin_phi),:]
    hist_phi = hist_eta_phi.project("phi", "pt")
    counts_phi, phi_bins, pt_bins = hist_phi.to_numpy(flow=False)
    counts_phi = np.squeeze(counts_phi)
    
    pt_centers = 0.5 * (pt_bins[:-1] + pt_bins[1:])
    
    weighted_counts_phi = np.sum(counts_phi * pt_centers, axis=1)
    weighted_variance_phi = np.sum(counts_phi * (pt_centers ** 2), axis=1)
    errs_phi = np.sqrt(weighted_variance_phi)
    errs_phi = np.where(weighted_counts_phi == 0, 0, errs_phi)
    
    if norm:
        scale_factor = np.sum(weighted_counts_phi) * np.diff(phi_bins)
        weighted_counts_phi /= scale_factor
        errs_phi /= scale_factor
    
    phi_centers = 0.5 * (phi_bins[1:] + phi_bins[:-1])
    l = ax0.errorbar(x=phi_centers, y=weighted_counts_phi, yerr=errs_phi, linestyle='')
    color = l[0].get_color()
    ax0.errorbar(x=phi_bins[:-1], y=weighted_counts_phi, drawstyle='steps-post', label=label, color=color)
            
    hist_ieta_iphi = hist_ieta_iphi[:,trigger,:,hist.rebin(rebin_iphi),:]
    hist_iphi = hist_ieta_iphi.project("iphi", "pt")
    counts_iphi, iphi_bins, pt_bins = hist_iphi.to_numpy(flow=False)
    counts_iphi = np.roll(counts_iphi, shift=-8, axis=0)
    
    weighted_counts_iphi = np.sum(counts_iphi * pt_centers, axis=1)
    weighted_variance_iphi = np.sum(counts_iphi * (pt_centers ** 2), axis=1)
    errs_iphi = np.sqrt(weighted_variance_iphi)
    errs_iphi = np.where(weighted_counts_iphi == 0, 0, errs_iphi)
    
    if norm:
        scale_factor = np.sum(weighted_counts_iphi) * np.diff(iphi_bins)
        weighted_counts_iphi /= scale_factor
        errs_iphi /= scale_factor
    
    iphi_bin_centers = np.arange(len(weighted_counts_iphi)) + 0.5
    iphi_bin_edges = np.arange(len(weighted_counts_iphi) + 1)
    iphi_labels = np.arange(len(weighted_counts_iphi))
    
    l = ax1.errorbar(x=iphi_bin_centers, y=weighted_counts_iphi, yerr=errs_iphi, linestyle='')
    color = l[0].get_color()
    ax1.errorbar(x=iphi_bin_edges[:-1], y=weighted_counts_iphi, drawstyle='steps-post', label=label, color=color)
    
    ax1.tick_params(axis='both', which='major', labelsize=12)
    ax1.set_xticks(iphi_bin_centers)
    ax1.set_xticklabels(iphi_labels)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(24, 10))
hist_name_eta_phi =  'lead_scoutingjet_eta_phi'
hist_name_ieta_iphi =  'lead_scoutingjet_ieta_iphi'
norm = True

for trigger in triggers_scouting:
    hist_current_eta_phi = getHist(hist_result_scouting, histpath_scouting+[hist_name_eta_phi])
    hist_current_ieta_iphi = getHist(hist_result_scouting, histpath_scouting+[hist_name_ieta_iphi])
    
    draw_hist1d_phi_ptweighted(hist_current_eta_phi, hist_current_ieta_iphi, ax[0], ax[1], trigger,
                               label=trigger_names_scouting[trigger], 
                               norm=norm, rebin_phi=1, rebin_iphi=1)
    
    
hep.cms.label('', ax=ax[0], data=True, lumi=None, year=title_scouting, com=13.6)
ax[0].set_xlim(-3.14,3.14)
ax[0].set_ylabel(f'Jet pT-Weighted Events{" [A.U.]" if norm else ""}', loc='top')
ax[0].set_xlabel(r'Lead Scouting Jet $\phi$', loc='right')
ax[0].legend();

hep.cms.label('', ax=ax[1], data=True, lumi=None, year=title_scouting, com=13.6)
ax[1].set_xlim(0,17)
ax[1].set_ylabel(f'Jet pT-Weighted Events{" [A.U.]" if norm else ""}', loc='top')
ax[1].set_xlabel(r'Lead Scouting Jet i$\phi$', loc='right')
ax[1].legend();

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(24, 10))
hist_name_eta_phi =  'lead_recojet_eta_phi'
hist_name_ieta_iphi =  'lead_recojet_ieta_iphi'
norm = True

for trigger in triggers_reco:
    hist_current_eta_phi = getHist(hist_result_reco, histpath_reco+[hist_name_eta_phi])
    hist_current_ieta_iphi = getHist(hist_result_reco, histpath_reco+[hist_name_ieta_iphi])
    
    draw_hist1d_phi_ptweighted(hist_current_eta_phi, hist_current_ieta_iphi, ax[0], ax[1], trigger,
                               label=trigger_names_reco[trigger], 
                               norm=norm, rebin_phi=1, rebin_iphi=1)
    
    
hep.cms.label('', ax=ax[0], data=True, lumi=None, year=title_reco, com=13.6)
ax[0].set_xlim(-3.14,3.14)
ax[0].set_ylabel(f'Jet pT-Weighted Events{" [A.U.]" if norm else ""}', loc='top')
ax[0].set_xlabel(r'Lead Reco Jet $\phi$', loc='right')
ax[0].legend();

hep.cms.label('', ax=ax[1], data=True, lumi=None, year=title_reco, com=13.6)
ax[1].set_xlim(0,17)
ax[1].set_ylabel(f'Jet pT-Weighted Events{" [A.U.]" if norm else ""}', loc='top')
ax[1].set_xlabel(r'Lead Reco Jet i$\phi$', loc='right')
ax[1].legend();

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(24, 10))
hist_name_eta_phi =  'lead_l1jet_eta_phi'
hist_name_ieta_iphi =  'lead_l1jet_ieta_iphi'
norm = True

for trigger in triggers_scouting:
    hist_current_eta_phi = getHist(hist_result_scouting, histpath_scouting+[hist_name_eta_phi])
    hist_current_ieta_iphi = getHist(hist_result_scouting, histpath_scouting+[hist_name_ieta_iphi])
    
    draw_hist1d_phi_ptweighted(hist_current_eta_phi, hist_current_ieta_iphi, ax[0], ax[1], trigger,
                               label=trigger_names_scouting[trigger], 
                               norm=norm, rebin_phi=1, rebin_iphi=1)
    
    
hep.cms.label('', ax=ax[0], data=True, lumi=None, year=title_scouting, com=13.6)
ax[0].set_xlim(-3.14,3.14)
ax[0].set_ylabel(f'Jet pT-Weighted Events{" [A.U.]" if norm else ""}', loc='top')
ax[0].set_xlabel(r'Lead L1Jet $\phi$', loc='right')
ax[0].legend();

hep.cms.label('', ax=ax[1], data=True, lumi=None, year=title_scouting, com=13.6)
ax[1].set_xlim(0,17)
ax[1].set_ylabel(f'Jet pT-Weighted Events{" [A.U.]" if norm else ""}', loc='top')
ax[1].set_xlabel(r'Lead L1Jet i$\phi$', loc='right')
ax[1].legend();

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(24, 10))
hist_name_eta_phi =  'lead_l1jet_eta_phi'
hist_name_ieta_iphi =  'lead_l1jet_ieta_iphi'
norm = True

for trigger in triggers_reco:
    hist_current_eta_phi = getHist(hist_result_reco, histpath_reco+[hist_name_eta_phi])
    hist_current_ieta_iphi = getHist(hist_result_reco, histpath_reco+[hist_name_ieta_iphi])
    
    draw_hist1d_phi_ptweighted(hist_current_eta_phi, hist_current_ieta_iphi, ax[0], ax[1], trigger,
                               label=trigger_names_reco[trigger], 
                               norm=norm, rebin_phi=1, rebin_iphi=1)
    
    
hep.cms.label('', ax=ax[0], data=True, lumi=None, year=title_reco, com=13.6)
ax[0].set_xlim(-3.14,3.14)
ax[0].set_ylabel(f'Jet pT-Weighted Events{" [A.U.]" if norm else ""}', loc='top')
ax[0].set_xlabel(r'Lead L1Jet $\phi$', loc='right')
ax[0].legend();

hep.cms.label('', ax=ax[1], data=True, lumi=None, year=title_reco, com=13.6)
ax[1].set_xlim(0,17)
ax[1].set_ylabel(f'Jet pT-Weighted Events{" [A.U.]" if norm else ""}', loc='top')
ax[1].set_xlabel(r'Lead L1Jet i$\phi$', loc='right')
ax[1].legend();

# non-pT Weighted phi distributions

In [ ]:
def draw_hist_phi(hist_phi, hist_iphi, ax0, ax1, trigger, label='', 
                  norm=False, rebin_phi=1, rebin_iphi=1):
    
    hist_phi = hist_phi[:,trigger,:,hist.rebin(rebin_phi),:]
    hist_phi = hist_phi.project("phi")
    counts_phi, phi_bins = hist_phi.to_numpy(flow=False)
    errs_phi = np.sqrt(counts_phi)
    errs_phi = np.where(counts_phi==0, 0, errs_phi)
    if norm:
        scale_factor = np.sum(counts_phi) * np.diff(phi_bins)
        counts_phi /= scale_factor
        errs_phi /= scale_factor
    
    phi_bin_centers = 0.5 * (phi_bins[1:] + phi_bins[:-1])
    l = ax0.errorbar(x=phi_bin_centers,y=counts_phi,yerr=errs_phi,linestyle='')
    color = l[0].get_color()
    ax0.errorbar(x=phi_bins[:-1],y=counts_phi,drawstyle='steps-post',label=label,color=color)
    
    hist_iphi = hist_iphi[:,trigger,:,hist.rebin(rebin_iphi),:]
    hist_iphi = hist_iphi.project("iphi")
    counts_iphi, iphi_bins = hist_iphi.to_numpy(flow=False)
    counts_iphi = np.roll(counts_iphi, shift=-8, axis=0)
    errs_iphi = np.sqrt(counts_iphi)
    errs_iphi = np.where(counts_iphi==0, 0, errs_iphi)
    if norm:
        scale_factor = np.sum(counts_iphi) * np.diff(iphi_bins)
        counts_iphi /= scale_factor
        errs_iphi /= scale_factor
    
    iphi_bin_centers = np.arange(len(counts_iphi)) + 0.5
    iphi_bin_edges = np.arange(len(counts_iphi) + 1)
    iphi_labels = np.arange(len(counts_iphi))
    
    l = ax1.errorbar(x=iphi_bin_centers, y=counts_iphi, yerr=errs_iphi, linestyle='')
    color = l[0].get_color()
    ax1.errorbar(x=iphi_bin_edges[:-1], y=counts_iphi, drawstyle='steps-post', label=label, color=color)
    
    ax1.tick_params(axis='both', which='major', labelsize=12)
    ax1.set_xticks(iphi_bin_centers)
    ax1.set_xticklabels(iphi_labels)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(24, 10))
hist_name_eta_phi =  'lead_scoutingjet_eta_phi'
hist_name_ieta_iphi =  'lead_scoutingjet_ieta_iphi'
norm = True

for trigger in triggers_scouting:
    hist_current_eta_phi = getHist(hist_result_scouting, histpath_scouting+[hist_name_eta_phi])
    hist_current_ieta_iphi = getHist(hist_result_scouting, histpath_scouting+[hist_name_ieta_iphi])
    
    draw_hist_phi(hist_current_eta_phi, hist_current_ieta_iphi, ax[0], ax[1], trigger,
                  label=trigger_names_scouting[trigger], 
                  norm=norm, rebin_phi=1, rebin_iphi=1)
    
    
hep.cms.label('', ax=ax[0], data=True, lumi=None, year=title_scouting, com=13.6)
ax[0].set_xlim(-3.14,3.14)
ax[0].set_ylabel(f'Events{" [A.U.]" if norm else ""}', loc='top')
ax[0].set_xlabel(r'Lead Scouting Jet $\phi$', loc='right')
ax[0].legend();

hep.cms.label('', ax=ax[1], data=True, lumi=None, year=title_scouting, com=13.6)
ax[1].set_xlim(0,17)
ax[1].set_ylabel(f'Events{" [A.U.]" if norm else ""}', loc='top')
ax[1].set_xlabel(r'Lead Scouting Jet i$\phi$', loc='right')
ax[1].legend();

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(24, 10))
hist_name_eta_phi =  'lead_recojet_eta_phi'
hist_name_ieta_iphi =  'lead_recojet_ieta_iphi'
norm = True

for trigger in triggers_reco:
    hist_current_eta_phi = getHist(hist_result_reco, histpath_reco+[hist_name_eta_phi])
    hist_current_ieta_iphi = getHist(hist_result_reco, histpath_reco+[hist_name_ieta_iphi])
    
    draw_hist_phi(hist_current_eta_phi, hist_current_ieta_iphi, ax[0], ax[1], trigger,
                  label=trigger_names_reco[trigger], 
                  norm=norm, rebin_phi=1, rebin_iphi=1)
    
    
hep.cms.label('', ax=ax[0], data=True, lumi=None, year=title_reco, com=13.6)
ax[0].set_xlim(-3.14,3.14)
ax[0].set_ylabel(f'Events{" [A.U.]" if norm else ""}', loc='top')
ax[0].set_xlabel(r'Lead Reco Jet $\phi$', loc='right')
ax[0].legend();

hep.cms.label('', ax=ax[1], data=True, lumi=None, year=title_reco, com=13.6)
ax[1].set_xlim(0,17)
ax[1].set_ylabel(f'Events{" [A.U.]" if norm else ""}', loc='top')
ax[1].set_xlabel(r'Lead Reco Jet i$\phi$', loc='right')
ax[1].legend();

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(24, 10))
hist_name_eta_phi =  'lead_l1jet_eta_phi'
hist_name_ieta_iphi =  'lead_l1jet_ieta_iphi'
norm = True

for trigger in triggers_scouting:
    hist_current_eta_phi = getHist(hist_result_scouting, histpath_scouting+[hist_name_eta_phi])
    hist_current_ieta_iphi = getHist(hist_result_scouting, histpath_scouting+[hist_name_ieta_iphi])
    
    draw_hist_phi(hist_current_eta_phi, hist_current_ieta_iphi, ax[0], ax[1], trigger,
                  label=trigger_names_scouting[trigger], 
                  norm=norm, rebin_phi=1, rebin_iphi=1)
    
    
hep.cms.label('', ax=ax[0], data=True, lumi=None, year=title_scouting, com=13.6)
ax[0].set_xlim(-3.14,3.14)
ax[0].set_ylabel(f'Events{" [A.U.]" if norm else ""}', loc='top')
ax[0].set_xlabel(r'Lead L1Jet $\phi$', loc='right')
ax[0].legend();

hep.cms.label('', ax=ax[1], data=True, lumi=None, year=title_scouting, com=13.6)
ax[1].set_xlim(0,17)
ax[1].set_ylabel(f'Events{" [A.U.]" if norm else ""}', loc='top')
ax[1].set_xlabel(r'Lead L1Jet i$\phi$', loc='right')
ax[1].legend();

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(24, 10))
hist_name_eta_phi =  'lead_l1jet_eta_phi'
hist_name_ieta_iphi =  'lead_l1jet_ieta_iphi'
norm = True

for trigger in triggers_reco:
    hist_current_eta_phi = getHist(hist_result_reco, histpath_reco+[hist_name_eta_phi])
    hist_current_ieta_iphi = getHist(hist_result_reco, histpath_reco+[hist_name_ieta_iphi])
    
    draw_hist_phi(hist_current_eta_phi, hist_current_ieta_iphi, ax[0], ax[1], trigger,
                  label=trigger_names_reco[trigger], 
                  norm=norm, rebin_phi=1, rebin_iphi=1)
    
    
hep.cms.label('', ax=ax[0], data=True, lumi=None, year=title_reco, com=13.6)
ax[0].set_xlim(-3.14,3.14)
ax[0].set_ylabel(f'Events{" [A.U.]" if norm else ""}', loc='top')
ax[0].set_xlabel(r'Lead L1Jet $\phi$', loc='right')
ax[0].legend();

hep.cms.label('', ax=ax[1], data=True, lumi=None, year=title_reco, com=13.6)
ax[1].set_xlim(0,17)
ax[1].set_ylabel(f'Events{" [A.U.]" if norm else ""}', loc='top')
ax[1].set_xlabel(r'Lead L1Jet i$\phi$', loc='right')
ax[1].legend();

# pT Weighted eta/phi maps

In [ ]:
def draw_hist2d_ptweighted(hist_eta_phi, hist_ieta_iphi, trigger_name, object_name="", 
                           dataset="Scouting", rebin_eta=1, rebin_phi=1, rebin_ieta=1, rebin_iphi=1):
    
    hist_eta_phi = hist_eta_phi[:,trigger,39:112,hist.rebin(rebin_phi),:]
    hist_eta_phi = hist_eta_phi[:, hist.rebin(rebin_eta), :, :]
    counts_eta_phi, _, eta_bins, phi_bins, pt_bins = hist_eta_phi.to_numpy(flow=False)
    counts_eta_phi = np.squeeze(counts_eta_phi)
    
    hist_ieta_iphi = hist_ieta_iphi[:,trigger,4:18,hist.rebin(rebin_iphi),:]
    hist_ieta_iphi = hist_ieta_iphi[:, hist.rebin(rebin_ieta), :, :]
    counts_ieta_iphi, _, ieta_bins, iphi_bins, pt_bins = hist_ieta_iphi.to_numpy(flow=False)
    counts_ieta_iphi = np.squeeze(counts_ieta_iphi)
    counts_ieta_iphi = np.roll(counts_ieta_iphi, shift=-8, axis=1)
    
    pt_centers = 0.5 * (pt_bins[:-1] + pt_bins[1:])
    
    h2_weighted_eta_phi = np.average(counts_eta_phi, axis=2, weights=pt_centers)

    h2_weighted_ieta_iphi = np.average(counts_ieta_iphi, axis=2, weights=pt_centers)
    ieta_bin_centers = np.arange(h2_weighted_ieta_iphi.shape[0]) + 4.5
    iphi_bin_centers = np.arange(h2_weighted_ieta_iphi.shape[1]) + 0.5
    ieta_bin_edges = np.arange(4,h2_weighted_ieta_iphi.shape[0] + 5)
    iphi_bin_edges = np.arange(h2_weighted_ieta_iphi.shape[1] + 1)
    iphi_labels = np.arange(h2_weighted_ieta_iphi.shape[1])
    ieta_labels = np.arange(h2_weighted_ieta_iphi.shape[0])+4
    
    fig, ax = plt.subplots(1, 2, figsize=(16, 8))

    ax[0].tick_params(axis='both', which='major', labelsize=12)
    ax[0].set_ylim(-3.14,3.14)
    x_label_0 = object_name + r" $\eta$"
    ax[0].set_xlabel(x_label_0)
    y_label_0 = object_name + r" $\phi$"
    ax[0].set_ylabel(y_label_0)
    
    mesh = ax[0].pcolormesh(eta_bins, phi_bins, h2_weighted_eta_phi.T, cmap="plasma", norm="log")
    
    
    ax[1].tick_params(axis='both', which='major', labelsize=12)
    ax[1].set_xticks(ieta_bin_centers)
    ax[1].set_xticklabels(ieta_labels)
    ax[1].set_xlim(4,18)
    ax[1].set_yticks(iphi_bin_centers)
    ax[1].set_yticklabels(iphi_labels)
    x_label_1 = object_name + r" i$\eta$"
    ax[1].set_xlabel(x_label_1)
    y_label_1 = object_name + r" i$\phi$"
    ax[1].set_ylabel(y_label_1)

    mesh = ax[1].pcolormesh(ieta_bin_edges, iphi_bin_edges, h2_weighted_ieta_iphi.T, cmap="plasma", norm="log")
    
    plt.suptitle(dataset + ": " + trigger_name)
    plt.subplots_adjust(hspace=0.5)

    fig.show()

In [ ]:
hist_name_eta_phi =  'lead_scoutingjet_eta_phi'
hist_name_ieta_iphi =  'lead_scoutingjet_ieta_iphi'

for trigger in triggers_scouting:
    
    trigger_name = trigger_names_scouting[trigger]
    hist_current_eta_phi = getHist(hist_result_scouting, histpath_scouting+[hist_name_eta_phi])
    hist_current_ieta_iphi = getHist(hist_result_scouting, histpath_scouting+[hist_name_ieta_iphi])
    
    draw_hist2d_ptweighted(hist_current_eta_phi, hist_current_ieta_iphi, 
                           trigger_name, object_name="Scouting Jet", dataset="Scouting", 
                           rebin_eta=1, rebin_phi=1, rebin_ieta=1, rebin_iphi=1)

In [ ]:
hist_name_eta_phi =  'lead_l1jet_eta_phi'
hist_name_ieta_iphi =  'lead_l1jet_ieta_iphi'

for trigger in triggers_scouting:
    
    trigger_name = trigger_names_scouting[trigger]
    hist_current_eta_phi = getHist(hist_result_scouting, histpath_scouting+[hist_name_eta_phi])
    hist_current_ieta_iphi = getHist(hist_result_scouting, histpath_scouting+[hist_name_ieta_iphi])
    
    draw_hist2d_ptweighted(hist_current_eta_phi, hist_current_ieta_iphi, 
                           trigger_name, object_name="L1 Jet", dataset="Scouting",
                           rebin_eta=1, rebin_phi=1, rebin_ieta=1, rebin_iphi=1)

In [ ]:
hist_name_eta_phi =  'lead_recojet_eta_phi'
hist_name_ieta_iphi =  'lead_recojet_ieta_iphi'

for trigger in triggers_reco:
    
    trigger_name = trigger_names_reco[trigger]
    hist_current_eta_phi = getHist(hist_result_reco, histpath_reco+[hist_name_eta_phi])
    hist_current_ieta_iphi = getHist(hist_result_reco, histpath_reco+[hist_name_ieta_iphi])
    
    draw_hist2d_ptweighted(hist_current_eta_phi, hist_current_ieta_iphi, 
                           trigger_name, object_name="Reco Jet", dataset="JetMET",
                           rebin_eta=1, rebin_phi=1, rebin_ieta=1, rebin_iphi=1)

In [ ]:
hist_name_eta_phi =  'lead_l1jet_eta_phi'
hist_name_ieta_iphi =  'lead_l1jet_ieta_iphi'

for trigger in triggers_reco:
    
    trigger_name = trigger_names_reco[trigger]
    hist_current_eta_phi = getHist(hist_result_reco, histpath_reco+[hist_name_eta_phi])
    hist_current_ieta_iphi = getHist(hist_result_reco, histpath_reco+[hist_name_ieta_iphi])
    
    draw_hist2d_ptweighted(hist_current_eta_phi, hist_current_ieta_iphi, 
                           trigger_name, object_name="L1 Jet", dataset="JetMET",
                           rebin_eta=1, rebin_phi=1, rebin_ieta=1, rebin_iphi=1)

In [ ]:
hist_name_eta_phi =  'lead_scoutingelectron_eta_phi'
hist_name_ieta_iphi =  'lead_scoutingelectron_ieta_iphi'

for trigger in triggers_scouting:
    
    trigger_name = trigger_names_scouting[trigger]
    hist_current_eta_phi = getHist(hist_result_scouting, histpath_scouting+[hist_name_eta_phi])
    hist_current_ieta_iphi = getHist(hist_result_scouting, histpath_scouting+[hist_name_ieta_iphi])
    
    draw_hist2d_ptweighted(hist_current_eta_phi, hist_current_ieta_iphi, 
                           trigger_name, object_name="Scouting Electron", dataset="Scouting", 
                           rebin_eta=1, rebin_phi=1, rebin_ieta=1, rebin_iphi=1)

In [ ]:
hist_name_eta_phi =  'lead_recoelectron_eta_phi'
hist_name_ieta_iphi =  'lead_recoelectron_ieta_iphi'

for trigger in triggers_reco:
    
    trigger_name = trigger_names_reco[trigger]
    hist_current_eta_phi = getHist(hist_result_reco, histpath_reco+[hist_name_eta_phi])
    hist_current_ieta_iphi = getHist(hist_result_reco, histpath_reco+[hist_name_ieta_iphi])
    
    draw_hist2d_ptweighted(hist_current_eta_phi, hist_current_ieta_iphi, 
                           trigger_name, object_name="Reco Electron", dataset="JetMET", 
                           rebin_eta=1, rebin_phi=1, rebin_ieta=1, rebin_iphi=1)

In [ ]:
hist_name_eta_phi =  'lead_scoutingphoton_eta_phi'
hist_name_ieta_iphi =  'lead_scoutingphoton_ieta_iphi'

for trigger in triggers_scouting:
    
    trigger_name = trigger_names_scouting[trigger]
    hist_current_eta_phi = getHist(hist_result_scouting, histpath_scouting+[hist_name_eta_phi])
    hist_current_ieta_iphi = getHist(hist_result_scouting, histpath_scouting+[hist_name_ieta_iphi])
    
    draw_hist2d_ptweighted(hist_current_eta_phi, hist_current_ieta_iphi, 
                           trigger_name, object_name="Scouting Photon", dataset="Scouting", 
                           rebin_eta=1, rebin_phi=1, rebin_ieta=1, rebin_iphi=1)

In [ ]:
hist_name_eta_phi =  'lead_recophoton_eta_phi'
hist_name_ieta_iphi =  'lead_recophoton_ieta_iphi'

for trigger in triggers_reco:
    
    trigger_name = trigger_names_reco[trigger]
    hist_current_eta_phi = getHist(hist_result_reco, histpath_reco+[hist_name_eta_phi])
    hist_current_ieta_iphi = getHist(hist_result_reco, histpath_reco+[hist_name_ieta_iphi])
    
    draw_hist2d_ptweighted(hist_current_eta_phi, hist_current_ieta_iphi, 
                           trigger_name, object_name="Reco Photon", dataset="JetMET", 
                           rebin_eta=1, rebin_phi=1, rebin_ieta=1, rebin_iphi=1)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(24, 10))
hist_name =  'lead_scoutingjet_eta'
norm = True

for trigger in triggers_scouting:
    hist_current = getHist(hist_result_scouting, histpath_scouting+[hist_name])
    draw_hist1d_eta_ptweighted(
        hist_current, 
        ax[0], 
        trigger, 
        label=trigger_names_scouting[trigger], 
        rebin=1, 
        norm=norm,
        use_ieta=True
    )
    
hep.cms.label('', ax=ax[0], data=True, lumi=None, year=title_scouting, com=13.6)
# ax[0].set_xlim(-4,4)
# ax[0].set_ylim(0,0.5)
ax[0].set_ylabel(f'Jet pT-Weighted Events{" [A.U.]" if norm else ""}', loc='top')
ax[0].set_xlabel(r'Lead ScoutingJet i$\eta$', loc='right')
# ax[0].set_yscale('log')
ax[0].legend();
ax[1].set_visible(False)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(24, 10))
hist_name =  'lead_recojet_eta'
norm = True

for trigger in triggers_reco:
    hist_current = getHist(hist_result_reco, histpath_reco+[hist_name])
    draw_hist1d_eta_ptweighted(
        hist_current, 
        ax[0], 
        trigger, 
        label=trigger_names_reco[trigger], 
        rebin=1, 
        norm=norm,
        use_ieta=True
    )
    
hep.cms.label('', ax=ax[0], data=True, lumi=None, year=title_reco, com=13.6)
# ax[0].set_xlim(-4,4)
# ax[0].set_ylim(0,0.5)
ax[0].set_ylabel(f'Jet pT-Weighted Events{" [A.U.]" if norm else ""}', loc='top')
ax[0].set_xlabel(r'Lead Reco Jet i$\eta$', loc='right')
# ax[0].set_yscale('log')
ax[0].legend();
ax[1].set_visible(False)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(24, 10))
hist_name =  'lead_scoutingjet_phi'
norm = True

for trigger in triggers_scouting:
    hist_current = getHist(hist_result_scouting, histpath_scouting+[hist_name])
    draw_hist1d_phi_ptweighted(
        hist_current, 
        ax[0], 
        trigger, 
        label=trigger_names_scouting[trigger], 
        rebin=1, 
        norm=norm,
        use_iphi=True
    )
    
hep.cms.label('', ax=ax[0], data=True, lumi=None, year=title_scouting, com=13.6)
#ax[0].set_xlim(-3.14,3.14)
# ax[0].set_ylim(0,0.5)
ax[0].set_ylabel(f'Jet pT-Weighted Events{" [A.U.]" if norm else ""}', loc='top')
ax[0].set_xlabel(r'Lead ScoutingJet $\phi$', loc='right')
# ax[0].set_yscale('log')
ax[0].legend();
ax[1].set_visible(False)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(24, 10))
hist_name =  'lead_recojet_phi'
norm = True

for trigger in triggers_reco:
    hist_current = getHist(hist_result_reco, histpath_reco+[hist_name])
    draw_hist1d_ptweighted(
        hist_current, 
        ax[0], 
        trigger, 
        label=trigger_names_reco[trigger], 
        rebin=1, 
        norm=norm
    )
    
hep.cms.label('', ax=ax[0], data=True, lumi=None, year=title_reco, com=13.6)
ax[0].set_xlim(-3.14,3.14)
# ax[0].set_ylim(0,0.5)
ax[0].set_ylabel(f'Jet pT-Weighted Events{" [A.U.]" if norm else ""}', loc='top')
ax[0].set_xlabel(r'Lead Reco Jet $\phi$', loc='right')
# ax[0].set_yscale('log')
ax[0].legend();
ax[1].set_visible(False)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(24, 10))
hist_name =  'lead_l1jet_eta'
norm = True

for trigger in triggers_scouting:
    hist_current = getHist(hist_result_scouting, histpath_scouting+[hist_name])
    draw_hist1d_ptweighted(
        hist_current, 
        ax[0], 
        trigger, 
        label=trigger_names_scouting[trigger], 
        rebin=3, 
        norm=norm
    )
    
hep.cms.label('', ax=ax[0], data=True, lumi=None, year=title_scouting, com=13.6)
ax[0].set_xlim(-4,4)
# ax[0].set_ylim(0,0.5)
ax[0].set_ylabel(f'Jet pT-Weighted Events{" [A.U.]" if norm else ""}', loc='top')
ax[0].set_xlabel(r'Lead L1Jet $\eta$', loc='right')
# ax[0].set_yscale('log')
ax[0].legend();
ax[1].set_visible(False)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(24, 10))
hist_name =  'lead_l1jet_eta'
norm = True

for trigger in triggers_reco:
    hist_current = getHist(hist_result_reco, histpath_reco+[hist_name])
    draw_hist1d_ptweighted(
        hist_current, 
        ax[0], 
        trigger, 
        label=trigger_names_reco[trigger], 
        rebin=3, 
        norm=norm
    )
    
hep.cms.label('', ax=ax[0], data=True, lumi=None, year=title_reco, com=13.6)
ax[0].set_xlim(-4,4)
# ax[0].set_ylim(0,0.5)
ax[0].set_ylabel(f'Jet pT-Weighted Events{" [A.U.]" if norm else ""}', loc='top')
ax[0].set_xlabel(r'Lead L1Jet $\eta$', loc='right')
# ax[0].set_yscale('log')
ax[0].legend();
ax[1].set_visible(False)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(24, 10))
hist_name =  'lead_l1jet_phi'
norm = True

for trigger in triggers_scouting:
    hist_current = getHist(hist_result_scouting, histpath_scouting+[hist_name])
    draw_hist1d_ptweighted(
        hist_current, 
        ax[0], 
        trigger, 
        label=trigger_names_scouting[trigger], 
        rebin=1, 
        norm=norm
    )
    
hep.cms.label('', ax=ax[0], data=True, lumi=None, year=title_scouting, com=13.6)
ax[0].set_xlim(-3.14,3.14)
# ax[0].set_ylim(0,0.5)
ax[0].set_ylabel(f'Jet pT-Weighted Events{" [A.U.]" if norm else ""}', loc='top')
ax[0].set_xlabel(r'Lead L1Jet $\phi$', loc='right')
# ax[0].set_yscale('log')
ax[0].legend();
ax[1].set_visible(False)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(24, 10))
hist_name =  'lead_l1jet_phi'
norm = True

for trigger in triggers_reco:
    hist_current = getHist(hist_result_reco, histpath_reco+[hist_name])
    draw_hist1d_ptweighted(
        hist_current, 
        ax[0], 
        trigger, 
        label=trigger_names_reco[trigger], 
        rebin=1, 
        norm=norm
    )
    
hep.cms.label('', ax=ax[0], data=True, lumi=None, year=title_reco, com=13.6)
ax[0].set_xlim(-3.14,3.14)
# ax[0].set_ylim(0,0.5)
ax[0].set_ylabel(f'Jet pT-Weighted Events{" [A.U.]" if norm else ""}', loc='top')
ax[0].set_xlabel(r'Lead L1Jet $\phi$', loc='right')
# ax[0].set_yscale('log')
ax[0].legend();
ax[1].set_visible(False)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(24, 10))
hist_name =  'lead_l1eg_eta'
norm = True

for trigger in triggers_scouting:
    hist_current = getHist(hist_result_scouting, histpath_scouting+[hist_name])
    draw_hist1d_ptweighted(
        hist_current, 
        ax[0], 
        trigger, 
        label=trigger_names_scouting[trigger], 
        rebin=2, 
        norm=norm
    )
    
hep.cms.label('', ax=ax[0], data=True, lumi=None, year=title_scouting, com=13.6)
ax[0].set_xlim(-4,4)
# ax[0].set_ylim(0,0.5)
ax[0].set_ylabel(f'Jet pT-Weighted Events{" [A.U.]" if norm else ""}', loc='top')
ax[0].set_xlabel(r'Lead L1EG $\eta$', loc='right')
# ax[0].set_yscale('log')
ax[0].legend();
ax[1].set_visible(False)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(24, 10))
hist_name =  'lead_l1eg_eta'
norm = True

for trigger in triggers_reco:
    hist_current = getHist(hist_result_reco, histpath_reco+[hist_name])
    draw_hist1d_ptweighted(
        hist_current, 
        ax[0], 
        trigger, 
        label=trigger_names_reco[trigger], 
        rebin=2, 
        norm=norm
    )
    
hep.cms.label('', ax=ax[0], data=True, lumi=None, year=title_reco, com=13.6)
ax[0].set_xlim(-4,4)
# ax[0].set_ylim(0,0.5)
ax[0].set_ylabel(f'Jet pT-Weighted Events{" [A.U.]" if norm else ""}', loc='top')
ax[0].set_xlabel(r'Lead L1EG $\eta$', loc='right')
# ax[0].set_yscale('log')
ax[0].legend();
ax[1].set_visible(False)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(24, 10))
hist_name =  'lead_l1eg_phi'
norm = True

for trigger in triggers_scouting:
    hist_current = getHist(hist_result_scouting, histpath_scouting+[hist_name])
    draw_hist1d_ptweighted(
        hist_current, 
        ax[0], 
        trigger, 
        label=trigger_names_scouting[trigger], 
        rebin=1, 
        norm=norm
    )
    
hep.cms.label('', ax=ax[0], data=True, lumi=None, year=title_scouting, com=13.6)
ax[0].set_xlim(-3.14,3.14)
# ax[0].set_ylim(0,0.5)
ax[0].set_ylabel(f'Jet pT-Weighted Events{" [A.U.]" if norm else ""}', loc='top')
ax[0].set_xlabel(r'Lead L1EG $\phi$', loc='right')
# ax[0].set_yscale('log')
ax[0].legend();
ax[1].set_visible(False)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(24, 10))
hist_name =  'lead_l1eg_phi'
norm = True

for trigger in triggers_reco:
    hist_current = getHist(hist_result_reco, histpath_reco+[hist_name])
    draw_hist1d_ptweighted(
        hist_current, 
        ax[0], 
        trigger, 
        label=trigger_names_reco[trigger], 
        rebin=1, 
        norm=norm
    )
    
hep.cms.label('', ax=ax[0], data=True, lumi=None, year=title_reco, com=13.6)
ax[0].set_xlim(-3.14,3.14)
# ax[0].set_ylim(0,0.5)
ax[0].set_ylabel(f'Jet pT-Weighted Events{" [A.U.]" if norm else ""}', loc='top')
ax[0].set_xlabel(r'Lead L1EG $\phi$', loc='right')
# ax[0].set_yscale('log')
ax[0].legend();
ax[1].set_visible(False)